<a href="https://colab.research.google.com/github/rickiepark/the-lm-book/blob/main/embedding_vs_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="display: flex; justify-content: center;">
    <div style="background-color: #f4f6f7; padding: 15px; width: 80%;">
        <table style="width: 100%">
            <tr>
                <td style="vertical-align: middle;">
                    <span style="font-size: 14px;">
                        A notebook for <a href="https://www.thelmbook.com" target="_blank" rel="noopener">The Hundred-Page Language Models Book</a> by Andriy Burkov<br><br>
                        Code repository: <a href="https://github.com/rickiepark/the-lm-book" target="_blank" rel="noopener">https://github.com/rickiepark/the-lm-book</a>
                    </span>
                </td>
                <td style="vertical-align: middle;">
                    <a href="https://www.thelmbook.com" target="_blank" rel="noopener">
                        <img src="https://thelmbook.com/img/book.png" width="80px" alt="The Hundred-Page Language Models Book">
                    </a>
                </td>
            </tr>
        </table>
    </div>
</div>

# 임베딩 층과 선형 층의 동등성

In [2]:
import torch
import torch.nn as nn

vocab_size = 10    # 시각화를 위해 작은 어휘 크기 사용
emb_dim = 4       # 작은 임베딩 차원
token_idx = 3     # 임베딩할 토큰의 인덱스

# 접근 방식 1: nn.Embedding 사용하여 인덱스를 밀집 벡터에 직접 매핑
embedding = nn.Embedding(vocab_size, emb_dim)

# 접근 방식 2: 동일한 효과를 얻기 위해 nn.Linear 사용
# Linear 층은 다음을 수행합니다: output = input @ weight.t() + bias
# 이 경우 bias=False 이므로: output = input @ weight.t()
linear = nn.Linear(vocab_size, emb_dim, bias=False)

# 임베딩 가중치를 Linear 층에 복사하고 전치
# 임베딩 가중치 크기: (vocab_size, emb_dim)
# Linear 가중치 크기: (emb_dim, vocab_size)  <- 전치에 유의
linear.weight.data = embedding.weight.data.t()

# Linear 층을 위한 원-핫 입력 생성 - token_idx를 제외한 모든 곳이 0
one_hot = torch.zeros(vocab_size)
one_hot[token_idx] = 1

# 임베딩 행렬에서 직접 임베딩 가져오기
# 내부적으로 embedding.weight[token_idx]가 수행됨
emb_output = embedding(torch.tensor([token_idx]))

# Linear 층의 경우: Linear 층은 (batch_size, input_dim) 형태를 예상하므로 배치 차원 추가
# 결과는: one_hot @ weight.t()이며, 이는 weight.t()에서 token_idx 행을 선택합니다.
linear_output = linear(one_hot.unsqueeze(0))

# 동일성을 확인하기 위해 출력 및 비교 결과 출력
print(f"임베딩 출력:\n{emb_output}\n")
print(f"Linear 출력:\n{linear_output}\n")
print(f"텐서가 동일한가요? {torch.equal(emb_output, linear_output)}")
print(f"텐서가 유사한가요? {torch.allclose(emb_output, linear_output)}")

# 숫자 정밀도 내에서 출력이 동일한지 확인
# 기본 허용 오차 사용: rtol=1e-5, atol=1e-8
# 공식: |x - y| ≤ atol + rtol * |y|
print(f"\n텐서 간 최대 차이: {(emb_output - linear_output).abs().max().item()}")

임베딩 출력:
tensor([[ 0.7639,  0.8223, -0.8133, -0.2121]], grad_fn=<EmbeddingBackward0>)

Linear 출력:
tensor([[ 0.7639,  0.8223, -0.8133, -0.2121]], grad_fn=<MmBackward0>)

텐서가 동일한가요? True
텐서가 유사한가요? True

텐서 간 최대 차이: 0.0
